 # 1. Setup Watson Machine Learning Connection

In [4]:
PROJECT_ID = 'find me in the Watson Studio UI'
API_KEY = 'Create me in the IBM Cloud UI'
LOCATION = 'https://us-south.ml.cloud.ibm.com'

In [5]:
# The code was removed by Watson Studio for sharing.

In [31]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "apikey": API_KEY,
    "url": LOCATION
}

wml_client = APIClient(wml_credentials)

wml_client.set.default_space('d15f4a9e-e1f5-4bc2-ae89-6606bd2b62c8') 


'SUCCESS'

In [7]:
wml_client.set.default_project(PROJECT_ID)

'SUCCESS'

# 2. Setting up DB2 Connection

In [8]:
import pandas as pd
import tensorflow as tf
import keras


In [ ]:
# 1. create a DB2 instance + upload your data
# 2. Create a "connection" in Watson Studio through the UI
# 3. Use the "insert to code" feature in this notebook

In [10]:
# The code was removed by Watson Studio for sharing.

/opt/conda/envs/Python-3.10/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,CLIENTNUM,ATTRITION_FLAG,CUSTOMER_AGE,GENDER,DEPENDENT_COUNT,EDUCATION_LEVEL,MARITAL_STATUS,INCOME_CATEGORY,CARD_CATEGORY,MONTHS_ON_BOOK,...,MONTHS_INACTIVE_12_MON,CONTACTS_COUNT_12_MON,CREDIT_LIMIT,TOTAL_REVOLVING_BAL,AVG_OPEN_TO_BUY,TOTAL_AMT_CHNG_Q4_Q1,TOTAL_TRANS_AMT,TOTAL_TRANS_CT,TOTAL_CT_CHNG_Q4_Q1,AVG_UTILIZATION_RATIO
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


# 3. Prepare the data

In [11]:
# convert all float columns to int
def convert_float_to_int(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('int64')
    return df

df=convert_float_to_int(df) 

In [12]:
# ignoring all non numeric features for now (just to keep it simple)
numeric_feature_names = ['CUSTOMER_AGE', 'DEPENDENT_COUNT', 'MONTHS_ON_BOOK',  'TOTAL_RELATIONSHIP_COUNT', 'MONTHS_INACTIVE_12_MON','CONTACTS_COUNT_12_MON','CREDIT_LIMIT','TOTAL_REVOLVING_BAL','AVG_OPEN_TO_BUY','TOTAL_AMT_CHNG_Q4_Q1','TOTAL_TRANS_CT','TOTAL_CT_CHNG_Q4_Q1','AVG_UTILIZATION_RATIO']
numeric_features = df[numeric_feature_names]
numeric_features.head()

,CUSTOMER_AGE,DEPENDENT_COUNT,MONTHS_ON_BOOK,TOTAL_RELATIONSHIP_COUNT,MONTHS_INACTIVE_12_MON,CONTACTS_COUNT_12_MON,CREDIT_LIMIT,TOTAL_REVOLVING_BAL,AVG_OPEN_TO_BUY,TOTAL_AMT_CHNG_Q4_Q1,TOTAL_TRANS_CT,TOTAL_CT_CHNG_Q4_Q1,AVG_UTILIZATION_RATIO
0,45,3,39,5,1,3,12691,777,11914,1,42,1,0
1,49,5,44,6,1,2,8256,864,7392,1,33,3,0
2,51,3,36,4,1,0,3418,0,3418,2,20,2,0
3,40,4,34,3,4,1,3313,2517,796,1,20,2,0
4,40,3,21,5,1,0,4716,0,4716,2,28,2,0


In [13]:
train_dataset = numeric_features.sample(frac=0.8, random_state=0)
test_dataset = numeric_features.drop(train_dataset.index)

train_target = train_dataset.pop('CREDIT_LIMIT')
test_target = test_dataset.pop('CREDIT_LIMIT')

# 4. Setting Up Model Monitoring

this will be the topic of a different presentation

In [15]:
#training_data_references = [
#                {
#                    "id": "credit_limit_prediction",
#                    "type": "container",
#                    "connection": {},
#                    "location": {
#                        "path": "customer_churn_db2"
#                    },
#                    "schema": {
#                        "id": "training_schema",
#                        "fields": [
#                            {"name": "I_SHOULD_WRITE_A_SCRIPT_TO_AUTOMATE_THIS", "nullable": True, "metadata": {}, "type": "string"}
#                        ]
#                    }
#                }
#            ]

In [16]:
#try:
#    from ibm_aigov_facts_client import AIGovFactsClient
#except:
#    !pip install -U ibm-aigov-facts-client
#    from ibm_aigov_facts_client import AIGovFactsClient
#    
#PROJECT_UID= os.environ['PROJECT_ID']
#CPD_URL=os.environ['RUNTIME_ENV_APSX_URL'][len('https://api.'):]
#CONTAINER_ID=PROJECT_ID
#CONTAINER_TYPE='project'
#EXPERIMENT_NAME='credit_limit_prediction'
#
#PROJECT_ACCESS_TOKEN=project.project_context.accessToken.replace('Bearer ','')
#
#facts_client = AIGovFactsClient(api_key=API_KEY,experiment_name=EXPERIMENT_NAME,container_type=CONTAINER_TYPE,container_id=CONTAINER_ID,set_as_current_experiment=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.6/547.6 kB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 93.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.5/182.5 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.3/82.3 kB 29.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 22.6 MB/s eta 0:00:00
  Created wheel for timeout-decorator: filename=timeout_decorator-0.5.0-py3-none-any.whl size=5027 sha256=d4fb626c527503d31cccc11cc5799d4eccd12d79edc75d498224402606ca5885
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/68/2f/bc/76f1192d474666d41ae6f09813fccbd00fe3f07e8261c4cff5
  Created wheel for databricks-cli: filename=databricks_cli-0.1

NameError: name 'project' is not defined

# 5. Do the actual Machine Learning

 https://www.tensorflow.org/tutorials/keras/regression

In [17]:
# not using a normalizer since it causes issues when saving the model https://github.com/keras-team/keras/issues/15348
def get_basic_model():
  model = tf.keras.Sequential([
    #normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=['accuracy'])
  return model

In [18]:
model = get_basic_model()
model.fit(train_dataset, train_target, epochs=100, batch_size=50, validation_split = 0.2)

Epoch 1/100
130/130 [==============================] - 1s 2ms/step - loss: 92964024.0000 - accuracy: 0.0000e+00 - val_loss: 33674036.0000 - val_accuracy: 0.0000e+00
Epoch 2/100
130/130 [==============================] - 0s 2ms/step - loss: 4973484.0000 - accuracy: 0.0000e+00 - val_loss: 55500.6328 - val_accuracy: 0.0000e+00
Epoch 3/100
130/130 [==============================] - 0s 1ms/step - loss: 30763.7930 - accuracy: 0.0000e+00 - val_loss: 17101.7910 - val_accuracy: 0.0000e+00
Epoch 4/100
130/130 [==============================] - 0s 1ms/step - loss: 14018.6797 - accuracy: 0.0000e+00 - val_loss: 12437.1250 - val_accuracy: 0.0000e+00
Epoch 5/100
130/130 [==============================] - 0s 2ms/step - loss: 11826.6650 - accuracy: 0.0000e+00 - val_loss: 11513.8975 - val_accuracy: 0.0000e+00
Epoch 6/100
130/130 [==============================] - 0s 2ms/step - loss: 11040.6494 - accuracy: 0.0000e+00 - val_loss: 10800.1211 - val_accuracy: 0.0000e+00
Epoch 7/100
130/130 [=================

In [19]:
test_dataset.iloc[0]

CUSTOMER_AGE                   45
DEPENDENT_COUNT                 3
MONTHS_ON_BOOK                 39
TOTAL_RELATIONSHIP_COUNT        5
MONTHS_INACTIVE_12_MON          1
CONTACTS_COUNT_12_MON           3
TOTAL_REVOLVING_BAL           777
AVG_OPEN_TO_BUY             11914
TOTAL_AMT_CHNG_Q4_Q1            1
TOTAL_TRANS_CT                 42
TOTAL_CT_CHNG_Q4_Q1             1
AVG_UTILIZATION_RATIO           0
Name: 0, dtype: int64

In [20]:
#TODO: make this prettier
index=10
print(model.predict(test_dataset.iloc[[1]]))
print(test_target.iloc[[1]])

1/1 [==============================] - 0s 67ms/step
[[8546.592]]
13    8547
Name: CREDIT_LIMIT, dtype: int64


# 6. Storing the model

WTF? https://stackoverflow.com/questions/53258236/how-to-save-keras-model-in-wml-repository-in-watson-studio

Do this: 
https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd3.5/notebooks/python_sdk/deployments/keras/Use%20Keras%20to%20recognize%20hand-written%20digits.ipynb

In [21]:
model_result_path = "keras_model.h5"
model.save(model_result_path)

In [22]:
!tar -zcvf keras_model.tgz keras_model.h5

keras_model.h5


In [23]:
! ls -la

total 64
drwxr-x--- 4 wsuser wscommon  4096 Dec  9 08:07 .
drwxrwx--- 1 wsuser wscommon  4096 Dec  9 08:00 ..
-rw-rw---- 1 wsuser wscommon 35432 Dec  9 08:07 keras_model.h5
-rw-rw---- 1 wsuser wscommon  4934 Dec  9 08:07 keras_model.tgz
drwxrwx--- 2 wsuser wscommon  4096 Dec  9 08:03 logs
drwxr-x--- 2 wsuser wscommon  4096 Dec  9 07:48 .virtual_documents


## 6.1. upload the model

In [24]:
! pip list | grep tensorflow # get the tensorflow version for the model metadata 

tensorflow                    2.9.2
tensorflow-addons             0.17.0
tensorflow-estimator          2.9.0
tensorflow-hub                0.12.0
tensorflow-io-gcs-filesystem  0.26.0
tensorflow-metadata           1.8.0
tensorflow-probability        0.17.0
tensorflow-text               2.9.0


In [37]:
sofware_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-22.2-py3.10")

metadata = {
            wml_client.repository.ModelMetaNames.NAME: 'credit limit model',
            wml_client.repository.ModelMetaNames.TYPE: 'tensorflow_2.9',
            wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = wml_client.repository.store_model(
    model="keras_model.tgz",
    meta_props=metadata)

## 6.2. get the model information programatically

In [38]:
import json

published_model_uid = wml_client.repository.get_model_id(published_model)
model_details = wml_client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "software_spec": {
      "id": "b56101f1-309d-549b-a849-eaa63f77b2fb",
      "name": "runtime-22.2-py3.10"
    },
    "type": "tensorflow_2.9"
  },
  "metadata": {
    "created_at": "2022-12-09T08:17:44.916Z",
    "id": "f945856c-bcef-4278-8d73-34bcade7bb7e",
    "modified_at": "2022-12-09T08:17:47.578Z",
    "name": "credit limit model",
    "owner": "IBMid-6650024HXN",
    "resource_key": "14da597f-4ea7-48a7-bd83-a0434aec7b95",
    "space_id": "d15f4a9e-e1f5-4bc2-ae89-6606bd2b62c8"
  },
  "system": {
    "warnings": []
  }
}


In [39]:
models_details = wml_client.repository.list_models()


------------------------------------  ------------------  ------------------------  --------------
ID                                    NAME                CREATED                   TYPE
f945856c-bcef-4278-8d73-34bcade7bb7e  credit limit model  2022-12-09T08:17:44.002Z  tensorflow_2.9
ad59ee55-7103-47cd-99a1-ff5a3a6cf1c2  credit limit model  2022-12-05T12:53:02.002Z  tensorflow_2.7
885da41b-b808-421e-9f0e-49cc0d5f6da6  credit limit model  2022-12-05T12:44:10.002Z  tensorflow_2.7
d2569dd9-1299-4846-af65-602d78432cca  credit limit model  2022-12-05T12:37:08.002Z  tensorflow_2.7
951eb7fb-8880-4a67-a066-63e2e26be830  credit limit model  2022-12-05T09:15:29.002Z  tensorflow_2.7
712dca3b-ff08-4e76-b7be-5bbedcd5851b  credit limit model  2022-12-05T07:56:16.002Z  tensorflow_2.7
ea57b539-8d0c-4aed-b1e7-5402813dde05  credit limit model  2022-12-05T06:13:18.002Z  tensorflow_2.7
------------------------------------  ------------------  ------------------------  --------------


In [41]:

metadata = {
    wml_client.deployments.ConfigurationMetaNames.NAME: "Deployment of external Keras model",
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = wml_client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'f945856c-bcef-4278-8d73-34bcade7bb7e' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='d3e78e70-b581-4f9b-86a5-4a1adb8de06c'
------------------------------------------------------------------------------------------------




In [42]:
deployment_uid = wml_client.deployments.get_id(created_deployment)

In [43]:
scoring_endpoint = wml_client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/ml/v4/deployments/d3e78e70-b581-4f9b-86a5-4a1adb8de06c/predictions


In [44]:
test_dataset.iloc[[1]]

,CUSTOMER_AGE,DEPENDENT_COUNT,MONTHS_ON_BOOK,TOTAL_RELATIONSHIP_COUNT,MONTHS_INACTIVE_12_MON,CONTACTS_COUNT_12_MON,TOTAL_REVOLVING_BAL,AVG_OPEN_TO_BUY,TOTAL_AMT_CHNG_Q4_Q1,TOTAL_TRANS_CT,TOTAL_CT_CHNG_Q4_Q1,AVG_UTILIZATION_RATIO
13,35,3,30,5,1,3,1666,6881,1,33,2,0


In [45]:
# TODO: make this prettier
scoring_payload = {"input_data": [{"values": test_dataset.iloc[[2]]}]}
predictions = wml_client.deployments.score(deployment_uid, scoring_payload)
predictions['predictions'][0]['values'][0][0][0] #pretty weird data structure :-P 

1437.07275390625